In [ ]:
import os
import json
import pandas as pd
import numpy as np
import json, pickle
import argparse
import gc
from tqdm.auto import tqdm
tqdm.pandas()
import regex as re
import json 
from glob import glob 
import re 
from nltk import word_tokenize as lib_tokenizer 
import string
from rank_bm25 import BM25Okapi
from pandarallel import pandarallel
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity
import numpy as np
pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=10)

In [ ]:
data = json.load(open("../za-data/zac2022_train_merged_final.json"))
data = [x for x in data['data'] if x["category"] == "FULL_ANNOTATION" and "wiki/" in x["answer"]]

In [ ]:
df_wiki = pd.read_json(path_or_buf = "../za-data/wikipedia_20220620_cleaned.jsonl", lines = True)

In [ ]:
def get_topk(query, topk = 100):
    tokenized_query = query.split()
    tfidf_query = tfidf_model[dictionary.doc2bow(tokenized_query)]
    scores = bm25_index[tfidf_query]
    top_n = np.argsort(scores)[::-1][:topk]
    titles = [df_wiki.title.values[i] for i in top_n]
    texts = [df_wiki.text.values[i] for i in top_n]
    # print(titles)
    return titles, texts, scores[top_n]

def get_topk_old(query, topk = 100):
    tokenized_query = query.split()
    scores = bm25.get_scores(tokenized_query)
    top_n = np.argsort(scores)[::-1][:topk]
    titles = [df_wiki.title.values[i] for i in top_n]
    texts = [df_wiki.text.values[i] for i in top_n]
    # print(titles)
    return titles, texts


dict_map = dict({})  
def word_tokenize(text): 
    global dict_map 
    words = text.split() 
    words_norm = [] 
    for w in words: 
        if dict_map.get(w, None) is None: 
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace('``', '"').replace("''", '"') 
        words_norm.append(dict_map[w]) 
    return words_norm 
 
def strip_answer_string(text): 
    text = text.strip() 
    while text[-1] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] != '(' and text[-1] == ')' and '(' in text: 
            break 
        if text[-1] == '"' and text[0] != '"' and text.count('"') > 1: 
            break 
        text = text[:-1].strip() 
    while text[0] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] == '"' and text[-1] != '"' and text.count('"') > 1: 
            break 
        text = text[1:].strip() 
    text = text.strip() 
    return text 
 
def strip_context(text): 
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text) 
    text = text.strip() 
    return text

def post_process(x):
    x = " ".join(word_tokenize(strip_context(x))).strip()
    x = x.replace("\n"," ")
    # x = x.replace(",","").replace(".","").replace("?","").replace("!","").replace(")","").replace("(","")
    x = "".join([i for i in x if i not in string.punctuation])
    x = " ".join(x.split()[:128])
    return x

In [ ]:
# # print(df_wiki.title.progress_apply)
df_wiki['title_lower'] = df_wiki['title'].progress_apply(lambda x: x.lower()).parallel_apply(post_process)
# df_wiki['context_lower'] = df_wiki['text'].progress_apply(lambda x: x.lower()).parallel_apply(post_process)

In [ ]:
title2idx = dict([(x.strip(),y) for x,y in zip(df_wiki.title, df_wiki.index.values)])

In [ ]:
tokenized_corpus = [doc.split() for doc in df_wiki['title_lower']] #simple tokenier
bm25_title = BM25Okapi(tokenized_corpus)

In [24]:
for x in data:
    answer = x['answer'].replace("wiki/","").replace("_"," ")
    if answer not in title2idx:
        for title in title2idx.keys():
            if answer.lower() == title.lower():
                x["answer"] = "wiki/"+title.replace(" ","_")
                print(x["answer"])

wiki/Cù_lao_Chàm
wiki/Nhà_Hán
wiki/Bán_đảo_Ả_Rập


In [ ]:
df = pd.DataFrame()
df["query"] = [post_process(x["short_candidate"]).lower().split() for x in data]

In [ ]:
def get_topk(query, topk=10):
    scores = bm25_title.get_scores(query)
    top_titles = np.argsort(scores)[::-1][:topk]
    return top_titles, scores[top_titles]
df["top_n"] = df["query"].parallel_apply(lambda x: get_topk(x)[0])

In [ ]:
topk = 10
total = 0
candidate_ids = []
true_ids = []
for i, x in tqdm(enumerate(data)):
    true_title = x["answer"].replace("_"," ").replace("wiki/","").strip()
    top_n = df.iloc[i].top_n 
    true_idx = title2idx[true_title]
    true_ids.append(true_idx)
    candidate_ids.append(list(top_n))

In [ ]:
for x, y in zip(candidate_ids, true_ids):
    if y not in x:
        x[-1] = y

In [ ]:
df = pd.DataFrame()
questions = []
answers = []
titles = []
candidates = []
labels = []
groups = []
for idx, (sample, true_idx, candidate_idxs) in tqdm(enumerate(zip(data, true_ids, candidate_ids))):
    assert true_idx in candidate_idxs
    question = sample['question']
    answer = sample['short_candidate']
    title = [df_wiki.title.values[i] for i in candidate_idxs]
    candidate = [df_wiki.text.values[i] for i in candidate_idxs]
    label = [1 if x == true_idx else 0 for x in candidate_idxs]
    
    questions.extend([question,]*len(candidate_idxs))
    answers.extend([answer,]*len(candidate_idxs))
    groups.extend([idx,]*len(candidate_idxs))
    titles.extend(title)
    candidates.extend(candidate)
    labels.extend(label)
    
df["question"] = questions 
df["answer"] = answers 
df["title"] = titles 
df["candidate"] = candidates 
df["label"] = labels 
df["group"] = groups 
df.candidate = df.candidate.apply(lambda x: " ".join(x.strip().split()))

In [ ]:
df.to_csv("./data/train_stage2_ranking.csv",index=False)

In [ ]:
!ls -halt data